**SA463A &#x25aa; Data Wrangling and Visualization &#x25aa; Fall 2020 &#x25aa; Foraker and Uhan**

# Project 5. COVID-19 in the US, revisited

This project will walk you through the steps necessary to recreate the datasets used in Project 3 from scratch. You will get some practice using the Pandas techniques you've learned so far this semester, learn a few new techniques, and get some exposure to the quirks of working with datasets out in the wild.

For your reference, the datasets from Project 3 have been included in the same folder as this notebook:

1. `data/covid_us_20201001.csv` contains data on COVID-19 cases and deaths for each county in the United States as of October 1, 2020. Each row contains the following data for each county:

| Column | Description | 
| :- | :- |
| `date` | Date of observation |
| `county_fips` | County FIPS code |
| `state` | State name |
| `county` | County name |
| `latitude` | Representative latitude coordinate |
| `longitude` | Representative longitude coordinate |
| `cases` | Cumulative total number of cases as of October 1, 2020 |
| `deaths` | Cumulative total number of deaths as of October 1, 2020 |
| `cases_per_100k` | Cumulative total number of cases as of October 1, 2020, per 100,000 people |
| `deaths_per_100k` | Cumulative total number of deaths as of October 1, 2020, per 100,000 people |
| `cases_week` | Number of new cases in the week ending on October 1, 2020 |
| `deaths_week` | Number of new deaths in the week ending on October 1, 2020 |
| `cases_week_per_100k` | Number of new cases in the week ending on October 1, 2020, per 100,000 people |
| `deaths_week_per_100k` | Number of new deaths in the week ending on October 1, 2020, per 100,000 people |


2. `data/covid_us_by_state_20201001.csv` contains data on COVID-19 cases and deaths for each state in the United States from February 1, 2020 to October 1, 2020. Each row contains the following data for each state and day:

| Column | Description | 
| :- | :- |
| `state` | State name |
| `date` | Date of observation |
| `cases` | Cumulative total number of cases as of October 1, 2020 |
| `deaths` | Cumulative total number of deaths as of October 1, 2020 |
| `cases_day` | Number of new cases on the date of observation |
| `deaths_day` | Number of new deaths on the date of observation |
| `cases_day_per_100k` | Number of cases on the date of observation, per 100,000 people |
| `deaths_day_per_100k` | Number of deaths on the date of observation, per 100,000 people |

Before we start, let's import Pandas.

In [ ]:
import pandas as pd

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Problem 1 &mdash; Population data

To compute the per capita metrics, such as `cases_per_100k` in `data/covid_us_20201001.csv`, we will use the 2019 estimated US county populations from the US Census.

[This web page](https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-total.html) contains different datasets related to county population. Near the bottom of the page, you can find [this CSV file](https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv) containing the estimated county populations from April, 1 2010 to July 1, 2019.

1. Download the CSV file and put it in the `data` subfolder of the folder containing this notebook.


2. Open the CSV file in Excel to get a sense of the dataset. We'll focus on the following columns:

| Column | Description |
| :- | :- |
| `STATE` | 2-digit state FIPS code |
| `COUNTY` | 3-digit county FIPS code |
| `STNAME` | State name | 
| `CTYNAME` | County name |
| `POPESTIMATE2019` | Estimated total population as of 7/1/2019 |


3. Using a single method chain, read and wrangle this dataset, and put the results into a DataFrame called `county_population_df`:<br><br>
    1. Use `pd.read_csv()` to read in the file. 
        - Due to some quirks in how the file is formatted, you'll need to use the `encoding='ISO-8859-1'` and `engine='python'` keyword arguments.
        - Note that by default, the FIPS codes will be read in as integers. As a result, the leading zeroes will be lost. This is OK, because as we'll see later, we'll be using data from other sources that have FIPS codes without leading zeroes.<br><br>
    
    2. Remove rows with `COUNTY` equal to `0`. These rows contain state-wide totals.<br><br>
    
    3. Create a new column called `county_fips` containing a "full" county FIPS code by combining `STATE` and `COUNTY` with the following formula:
    
        $$
        1000 \times (\text{state FIPS code}) + (\text{county FIPS code})
        $$<br>

    4. Keep only the following columns: `county_fips`, `STNAME`, `CTYNAME`, and `POPESTIMATE2019`. We won't need the others.<br><br>
    
    5. Rename the columns: `STNAME` &rarr; `state`, `CTYNAME` &rarr; `county`, `POPESTIMATE2019` &rarr; `population`.
    

4. Display the first 5 rows of `county_population_df`. You should see something like this:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>county_fips</th>
      <th>state</th>
      <th>county</th>
      <th>population</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>1001</td>
      <td>Alabama</td>
      <td>Autauga County</td>
      <td>55869</td>
    </tr>
    <tr>
      <th>2</th>
      <td>1003</td>
      <td>Alabama</td>
      <td>Baldwin County</td>
      <td>223234</td>
    </tr>
    <tr>
      <th>3</th>
      <td>1005</td>
      <td>Alabama</td>
      <td>Barbour County</td>
      <td>24686</td>
    </tr>
    <tr>
      <th>4</th>
      <td>1007</td>
      <td>Alabama</td>
      <td>Bibb County</td>
      <td>22394</td>
    </tr>
    <tr>
      <th>5</th>
      <td>1009</td>
      <td>Alabama</td>
      <td>Blount County</td>
      <td>57826</td>
    </tr>
  </tbody>
</table>

## Problem 2 &mdash; Geographic data

To obtain the latitude and longitude of each county, as in the CSV file `data/covid_us_20201001.csv`, we will again use data from the US Census.

[This web page](https://www.census.gov/geographies/reference-files/time-series/geo/gazetteer-files.html) contains the US Gazetteer Files, which provide a listing of all geographic areas for selected geographic area types, such as counties. Somewhere in the middle of the web page, you can find [this ZIP file](https://www2.census.gov/geo/docs/maps-data/data/gazetteer/2019_Gazetteer/2019_Gaz_counties_national.zip) containing the National Counties Gazetteer File.

1. Download the ZIP file and put it in the `data` subfolder of the folder containing this notebook.


2. Unzip the ZIP file. You should see that there is one file in the ZIP file package, `2019_Gaz_counties_national.txt`. Open the file in Excel as a **tab-delimited** file to get a sense of the dataset. We'll focus on the following columns:

| Column | Description |
| :- | :- |
| `GEOID` | "Full" county FIPS code |
| `INTPTLAT` | Representative latitude coordinates |
| `INTPTLONG` | Representative longitude coordinates |

In Excel, note that the name of `INPTLONG` has a lot of *trailing whitespace* (extra spaces at the end). You'll take care of this when we read in the file with Pandas.


3. Using a single method chain, read and wrangle this dataset, and put the results into a DataFrame called `county_location_df`:<br><br>
    1. Use `pd.read_csv()` to read in the file. 
        - We can read tab-delimited files with `pd.read_csv()` using the `delimiter='\t'` keyword argument.<br><br>
        
    2. Sanitize the column names by removing all leading and trailing whitespace (extra spaces at the beginning and the end). You can do this with the `.rename()` DataFrame method. Instead of passing a dictionary to the keyword argument `columns=...`, you can pass a *function* that acts on each column name string. In this case, we can use the [`.strip()` Python string method](https://docs.python.org/3/library/stdtypes.html#str.strip) to remove leading and trailing whitespace from each column name, like this:
    
        ```python
        ...
        .rename(columns=lambda s: s.strip())
        ...
        ```
   
   3. Keep only the following columns: `GEOID`, `INTPLAT`, `INTPTLONG`. We won't need the others.<br><br>
   
   4. Rename the columns: `GEOID` &rarr; `county_fips`, `INTPTLAT` &rarr; `latitude`, `INTPTLONG` &rarr; `longitude`. 
       - We'll rename any column containing the county FIPS codes to `county_fips`, to make merging DataFrames easier later on.
   

4. Display the first 5 rows of `county_location_df`. You should see something like this:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>county_fips</th>
      <th>latitude</th>
      <th>longitude</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>1001</td>
      <td>32.532237</td>
      <td>-86.646440</td>
    </tr>
    <tr>
      <th>1</th>
      <td>1003</td>
      <td>30.659218</td>
      <td>-87.746067</td>
    </tr>
    <tr>
      <th>2</th>
      <td>1005</td>
      <td>31.870253</td>
      <td>-85.405104</td>
    </tr>
    <tr>
      <th>3</th>
      <td>1007</td>
      <td>33.015893</td>
      <td>-87.127148</td>
    </tr>
    <tr>
      <th>4</th>
      <td>1009</td>
      <td>33.977358</td>
      <td>-86.566440</td>
    </tr>
  </tbody>
</table>


## Problem 3 &mdash; Number of cases

To obtain the number of COVID-19 cases in each county, we will use the data from [this USA Facts web page](https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/). Near the bottom of the web page, you can download [this CSV file](https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv) containing the cumulative number of COVID-19 cases in each county since January 22, 2020.


1. Download the CSV file and put it in the `data` subfolder of the folder containing this notebook.


2. Open the CSV file in Excel to get a sense of the dataset. We will focus on the following columns:
    
| Column | Description |
| :- | :- |
| `countyFIPS` | County FIPS code |
| `mm/dd/yy` | Number of cases on date mm/dd/yy |

Note that there are some special values of `countyFIPS`: `0` for state-wide unallocated cases, `1` for New York City unallocated cases.


3. Using a single method chain, read and wrangle this dataset and put the results into a DataFrame called `cases_df`:<br><br>
    1. Use `pd.read_csv()` to read in the file.<br><br>
    
    2. Drop the following columns: `stateFIPS`, `State`, and `County Name`. We won't need them.<br><br>
    
    3. Rename the `countyFIPS` column to `county_fips`.<br><br>
    
    4. Keep only the rows with `county_fips` strictly greater than 1000.
    

4. Display the first 5 rows of `cases_df`. You should see something like this:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>county_fips</th>
      <th>1/22/20</th>
      <th>1/23/20</th>
      <th>1/24/20</th>
      <th>1/25/20</th>
      <th>1/26/20</th>
      <th>1/27/20</th>
      <th>1/28/20</th>
      <th>1/29/20</th>
      <th>1/30/20</th>
      <th>1/31/20</th>
      <th>...</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>1001</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>...</td>
    </tr>
    <tr>
      <th>2</th>
      <td>1003</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>...</td>
    </tr>
    <tr>
      <th>3</th>
      <td>1005</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>...</td>
    </tr>
    <tr>
      <th>4</th>
      <td>1007</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>...</td>
    </tr>
    <tr>
      <th>5</th>
      <td>1009</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>...</td>
    </tr>
  </tbody>
</table>

## Problem 4 &mdash; Number of cases, cont.

Let's tidy the data in `cases_df` by pivoting it to long form, so that the dates of observation are in a column.

1. Using a single method chain, create a new DataFrame `cases_long_df`:<br><br>
    1. Pivot `cases_df` from wide to long form. Put the dates into a column named `date`, and the numbers of cases into a column named `cases`.<br><br>
    2. Convert the values in the column `date` in the pivoted DataFrame to proper Python datetime objects with `pd.to_datetime()`. Overwrite the existing `date` column, like this:
    
    ```python
    ...
    .assign(
            date=lambda x: pd.to_datetime(x['date'])
    )
    ...
    ```
    
2. Display the first 5 rows of `cases_long_df`. You should see something like this:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>county_fips</th>
      <th>date</th>
      <th>cases</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>1001</td>
      <td>2020-01-22</td>
      <td>0</td>
    </tr>
    <tr>
      <th>1</th>
      <td>1003</td>
      <td>2020-01-22</td>
      <td>0</td>
    </tr>
    <tr>
      <th>2</th>
      <td>1005</td>
      <td>2020-01-22</td>
      <td>0</td>
    </tr>
    <tr>
      <th>3</th>
      <td>1007</td>
      <td>2020-01-22</td>
      <td>0</td>
    </tr>
    <tr>
      <th>4</th>
      <td>1009</td>
      <td>2020-01-22</td>
      <td>0</td>
    </tr>
  </tbody>
</table>

## Problem 5 &mdash; Number of deaths

To obtain the number of COVID-19 deaths in each county, we will use data also from USA Facts. Near the bottom of [this web page (same as the one linked above)](https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/), you can download [this CSV file](https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_deaths_usafacts.csv) containing the cumulative number of COVID-19 deaths in each county since January 22, 2020.

Use the same instructions as in Problem 3 to read and wrangle this dataset. Put the results in a DataFrame called `deaths_df`. Display the first 5 rows of `deaths_df`.

## Problem 6 &mdash; Number of deaths, cont.

Use the same instructions as in Problem 4 to tidy the deaths data from USA Facts. Put the results into a DataFrame called `deaths_long_df`, with the number of deaths in a column called `deaths`. Display the first 5 rows of `deaths_long_df`.

## Problem 7 &mdash; Merge the data together

Now it's finally time to merge the population data, geographic data, and COVID-19 cases data. 


1. Using a single method chain, merge the datasets together and put the results in a DataFrame called `merged_df`:<br><br>
    
    1. Start with `cases_long_df` as the "left" DataFrame and `deaths_long_df` as the "right" DataFrame. Merge on `county_fips` and `date`, using the keys from the "left" only.<br><br>
    
    2. Take the resulting DataFrame from part A as the "left", and `county_location_df` as the "right". Merge on `county_fips`, using keys from the "left" only.<br><br>
    
    3. Take the resulting DataFrame from part B as the "left", and `county_population_df` as the "right". Merge on `county_fips`, using keys from the "left" only.<br><br>
    

       
2. Display the first 5 rows of `merged_df`. You should see something like this: 

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>county_fips</th>
      <th>date</th>
      <th>cases</th>
      <th>deaths</th>
      <th>latitude</th>
      <th>longitude</th>
      <th>state</th>
      <th>county</th>
      <th>population</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>1001</td>
      <td>2020-01-22</td>
      <td>0</td>
      <td>0</td>
      <td>32.532237</td>
      <td>-86.646440</td>
      <td>Alabama</td>
      <td>Autauga County</td>
      <td>55869.0</td>
    </tr>
    <tr>
      <th>1</th>
      <td>1003</td>
      <td>2020-01-22</td>
      <td>0</td>
      <td>0</td>
      <td>30.659218</td>
      <td>-87.746067</td>
      <td>Alabama</td>
      <td>Baldwin County</td>
      <td>223234.0</td>
    </tr>
    <tr>
      <th>2</th>
      <td>1005</td>
      <td>2020-01-22</td>
      <td>0</td>
      <td>0</td>
      <td>31.870253</td>
      <td>-85.405104</td>
      <td>Alabama</td>
      <td>Barbour County</td>
      <td>24686.0</td>
    </tr>
    <tr>
      <th>3</th>
      <td>1007</td>
      <td>2020-01-22</td>
      <td>0</td>
      <td>0</td>
      <td>33.015893</td>
      <td>-87.127148</td>
      <td>Alabama</td>
      <td>Bibb County</td>
      <td>22394.0</td>
    </tr>
    <tr>
      <th>4</th>
      <td>1009</td>
      <td>2020-01-22</td>
      <td>0</td>
      <td>0</td>
      <td>33.977358</td>
      <td>-86.566440</td>
      <td>Alabama</td>
      <td>Blount County</td>
      <td>57826.0</td>
    </tr>
  </tbody>
</table>

## Problem 8 &mdash; Compute additional metrics

Now that we have a DataFrame that contains the population, geographic, and COVID-19 data, we can compute some additional metrics.

1. Using a single method chain, produce a new DataFrame called `df` with the following new metrics:<br><br>
    1. **Cases per 100,000.** Create a new variable called `cases_per_100k` containing the number of cases per 100,000 for each observation (i.e., each `county_fips`-`date` pair).<br><br>
    2. **Deaths per 100,000.** Create a new variable called `deaths_per_100k` containing the number of deaths per 100,000 for each observation.<br><br>
    3. **New cases in the last week.** To do this, first make sure `merged_df` is sorted by `county_fips` and `date`. Then create a new variable called `cases_week` using split-apply-combine: group the data by `county_fips`, then use `.transform()` to apply `.diff(7)` to each group's `cases` column. Here is the [documentation for `.diff()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.diff.html).<br><br>
    4. **New deaths in the last week.** Do the same as in part C, except call the new variable `deaths_week`, and perform split-apply-transform on the `deaths` column.<br><br>
    5. **New cases in the last week per 100,000.** Create a new variable called `cases_week_per_100k` containing the number of new cases in the last week per 100,000 for each observation.<br><br>
    4. **New deaths in the last week per 100,000.** Create a new variable called `deaths_week_per_100k` containing the number of deaths in the last week per 100,000 for each observation.<br><br>
    
2. Display the first 10 rows of `df`, so you can see if `cases_week` and `deaths_week` were computed correctly. You should see something that looks like this:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>county_fips</th>
      <th>date</th>
      <th>cases</th>
      <th>deaths</th>
      <th>latitude</th>
      <th>longitude</th>
      <th>state</th>
      <th>county</th>
      <th>population</th>
      <th>cases_per_100k</th>
      <th>deaths_per_100k</th>
      <th>cases_week</th>
      <th>deaths_week</th>
      <th>cases_week_per_100k</th>
      <th>deaths_week_per_100k</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>1001</td>
      <td>2020-01-22</td>
      <td>0</td>
      <td>0</td>
      <td>32.532237</td>
      <td>-86.64644</td>
      <td>Alabama</td>
      <td>Autauga County</td>
      <td>55869.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>3144</th>
      <td>1001</td>
      <td>2020-01-23</td>
      <td>0</td>
      <td>0</td>
      <td>32.532237</td>
      <td>-86.64644</td>
      <td>Alabama</td>
      <td>Autauga County</td>
      <td>55869.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>6288</th>
      <td>1001</td>
      <td>2020-01-24</td>
      <td>0</td>
      <td>0</td>
      <td>32.532237</td>
      <td>-86.64644</td>
      <td>Alabama</td>
      <td>Autauga County</td>
      <td>55869.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>9432</th>
      <td>1001</td>
      <td>2020-01-25</td>
      <td>0</td>
      <td>0</td>
      <td>32.532237</td>
      <td>-86.64644</td>
      <td>Alabama</td>
      <td>Autauga County</td>
      <td>55869.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>12576</th>
      <td>1001</td>
      <td>2020-01-26</td>
      <td>0</td>
      <td>0</td>
      <td>32.532237</td>
      <td>-86.64644</td>
      <td>Alabama</td>
      <td>Autauga County</td>
      <td>55869.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>15720</th>
      <td>1001</td>
      <td>2020-01-27</td>
      <td>0</td>
      <td>0</td>
      <td>32.532237</td>
      <td>-86.64644</td>
      <td>Alabama</td>
      <td>Autauga County</td>
      <td>55869.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>18864</th>
      <td>1001</td>
      <td>2020-01-28</td>
      <td>0</td>
      <td>0</td>
      <td>32.532237</td>
      <td>-86.64644</td>
      <td>Alabama</td>
      <td>Autauga County</td>
      <td>55869.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>22008</th>
      <td>1001</td>
      <td>2020-01-29</td>
      <td>0</td>
      <td>0</td>
      <td>32.532237</td>
      <td>-86.64644</td>
      <td>Alabama</td>
      <td>Autauga County</td>
      <td>55869.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
    </tr>
    <tr>
      <th>25152</th>
      <td>1001</td>
      <td>2020-01-30</td>
      <td>0</td>
      <td>0</td>
      <td>32.532237</td>
      <td>-86.64644</td>
      <td>Alabama</td>
      <td>Autauga County</td>
      <td>55869.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
    </tr>
    <tr>
      <th>28296</th>
      <td>1001</td>
      <td>2020-01-31</td>
      <td>0</td>
      <td>0</td>
      <td>32.532237</td>
      <td>-86.64644</td>
      <td>Alabama</td>
      <td>Autauga County</td>
      <td>55869.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
    </tr>
  </tbody>
</table>

## Problem 9 &mdash; County-level data on October 1

Now we're ready to recreate `data/covid_us_20201001.csv`.

1. Using a single method chain, create a new DataFrame called `county_oct1_df` based on `df`:<br><br>
    1. Keep only the rows corresponding to October 1, 2020.<br><br>
    2. Sort the rows by `county_fips`.<br><br>
    2. Reorder the columns in this order: `date`, `county_fips`, `state`, `county`, `latitude`, `longitude`, `population`, `cases`, `deaths`, `cases_per_100k`, `deaths_per_100k`, `cases_week`, `deaths_week`, `cases_week_per_100k`, `deaths_week_per_100k`<br><br>

2. Write this DataFrame to a CSV file called `county_oct1_df.csv`. You can do this with the `.to_csv()` DataFrame method, like this:

    ```python
    county_oct1_df.to_csv('county_oct1_df.csv', index=False)
    ```
    
    The `index=False` keyword argument prevents the index being included in the output file.

## Problem 10 &mdash; State-level data between February 1 and October 1

To recreate `data/covid_us_by_state_20201001.csv`, we'll need to do a little more work.


1. Using a single method chain, create a new DataFrame called `state_feb1_oct1_df` that contains the new daily cases and deaths in each state between February 1 and October 1:<br><br>
    1. Keep only the rows of `df` corresponding to dates between January 31, 2020 and October 1, 2020, inclusive.<br><br>
    2. Use split-apply-combine to create a DataFrame that contains the total number of cases, the total number of deaths, and the total population for each state on each date. Name these columns `cases`, `deaths`, and `population`, respectively.<br><br>
    3. Ensure that the resulting DataFrame is sorted by `state` and `date`.<br><br>
    4. Use split-apply-combine to create a new variable called `cases_day` containing the number of new cases in the past day: group the data by `state` (but not `date`), then use `.transform()` to apply `.diff(1)` to each group's `cases` column.<br><br>
    5. Do the same for deaths, creating a new variable called `deaths_day` containing the number of new deaths in the past day:  group the data by `state` (but not `date`), then use `.transform()` to apply `.diff(1)` to each group's `deaths` column.<br><br>
    6. Create a new variable called `cases_day_per_100k` containing the number of cases per 100,000.<br><br>
    7. Create a new variable called `deaths_day_per_100k` containing the number of deaths per 100,000.<br><br>
    8. Drop any row with missing data.<br><br>
    9. Keep only the following columns, in this order: `state`, `date`, `cases`, `deaths`, `cases_day`, `deaths_day`, `cases_day_per_100k`, `deaths_day_per_100k`.
    
    
2. Write this DataFrame to a CSV file called `state_feb1_oct1_df.csv`.    

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Grading rubric

| Problem |                          | Points  |
| :-      | :-                       | -:      |
| 1       | Item 3A                  | 1       |
|         | Item 3B                  | 1       |
|         | Item 3C                  | 2       |
|         | Item 3D                  | 1       |
|         | Item 3E                  | 1       |
|         | Item 4                   | 1       |
|         | Code runs without errors | 3       |
| 2       | Item 3A                  | 1       |
|         | Item 3B                  | 1       |
|         | Item 3C                  | 1       |
|         | Item 3D                  | 1       |
|         | Item 4                   | 1       |
|         | Code runs without errors | 2       |
| 3       | Item 3A                  | 1       |
|         | Item 3B                  | 1       |
|         | Item 3C                  | 1       |
|         | Item 3D                  | 1       |
|         | Item 4                   | 1       |
|         | Code runs without errors | 2       |
| 4       | Item 1A                  | 3       |
|         | Item 1B                  | 2       |
|         | Item 2                   | 1       |
|         | Code runs without errors | 3       |
| 5       | Item 3A                  | 1       |
|         | Item 3B                  | 1       |
|         | Item 3C                  | 1       |
|         | Item 3D                  | 1       |
|         | Item 4                   | 1       |
|         | Code runs without errors | 2       |
| 6       | Item 1A                  | 3       |
|         | Item 1B                  | 2       |
|         | Item 2                   | 1       |
|         | Code runs without errors | 3       |
| 7       | Item 1A                  | 2       |
|         | Item 1B                  | 2       |
|         | Item 1C                  | 2       |
|         | Item 2                   | 1       |
|         | Code runs without errors | 3       |
| 8       | Item 1A                  | 1       |
|         | Item 1B                  | 1       |
|         | Item 1C                  | 2       |
|         | Item 1D                  | 2       |
|         | Item 1E                  | 1       |
|         | Item 1F                  | 1       |
|         | Item 2                   | 1       |
|         | Code runs without errors | 4       |
| 9       | Item 1A                  | 1       |
|         | Item 1B                  | 1       |
|         | Item 1C                  | 1       |
|         | Item 2                   | 2       |
|         | Code runs without errors | 3       |
| 10      | Item 1A                  | 1       |
|         | Item 1B                  | 2       |
|         | Item 1C                  | 1       |
|         | Item 1D                  | 2       |
|         | Item 1E                  | 2       |
|         | Item 1F                  | 1       |
|         | Item 1G                  | 1       |
|         | Item 1H                  | 1       |
|         | Item 1I                  | 1       |
|         | Item 2                   | 2       |
|         | Code runs without errors | 6       |
|         | **Total**                | **100** |